In [1]:
# 1. Import libraries:
%matplotlib inline
import os
import quandl
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as spo
from scipy.stats import kurtosis, skew
import seaborn as sns
# from financial_data import *
import tensorflow as tf
mlp.style.use('seaborn')
quandl.save_key('HtwBLPt3k37yZHTvy15K')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 20)

/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_2244/4061755221.py:16: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  mlp.style.use('seaborn')


In [2]:
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500 = pd.read_html(sp_url, header=0)[0]
sp500.head()



,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [3]:
# Correct invalid dates:
sp500.loc[sp500[sp500['Date added']=='1983-11-30 (1957-03-04)'].index,'Date added'] = '1983-11-30'
sp500.loc[sp500[sp500['Date added']=='2001?'].index,'Date added'] = '2001-01-01'
# Filter firms that entered the index after December 2015:
sp500['Date added'] = pd.to_datetime(sp500['Date added'],format='%Y-%m-%d')
sp500 = sp500[sp500['Date added']<'2007-01-01']
print("The number of stocks in the universe is:", sp500.shape[0])

The number of stocks in the universe is: 241


In [4]:
n_stocks = 1
np.random.seed(1792)
universe_tickers = sp500['Symbol'].unique()
tickers = list(np.random.choice(universe_tickers,replace=False,size=n_stocks))
print(tickers)
# sp500[sp500['Symbol'].isin(portfolio_tickers)]

['DIS']


In [5]:
# tickers = ['AAPL', 'GOOGL', 'MSFT','SPY']
# start_date='2007-01-01'
# end_date='2022-12-31'
# columns = ['Adj Close', 'Volume']
# # Download data
# data = yf.download(tickers, start=start_date, end=end_date)[columns]

# # 'data' will be a Pandas DataFrame containing the historical data for the specified tickers
# print(data)


def download_data(tickers, start_date, end_date, columns):
    data = yf.download(tickers, start=start_date, end=end_date)
    data=data[columns]
    return data

# Define the tickers, start date, end date, and columns you want to download
# tickers = ['AAPL', 'GOOGL', 'MSFT']
start_date = '2007-01-01'
end_date = '2022-12-31'
columns = [ 'Adj Close', 'Volume','Open','High','Low']

# Download the data
data = download_data(tickers, start_date, end_date, columns)
# print(data)


def one_lvl_colnames(df,cols,tickers):
    """This function changes a multi-level column indexation into a one level
    column indexation

    Inputs:
    -------
    df (pandas Dataframe): dataframe with the columns whose indexation will be 
        flattened.
    tickers (list|string): list/string with the tickers (s) in the data frame df.
    cols (list|string): list/string with the name of the columns (e.g. 'Adj Close',
        'High', 'Close', etc.) that are in the dataframe df.
    
    Ouputs:
    -------
    df (pandas Dataframe): dataframe with the same information as df, but 
        with one level of indexation.
    """
    
    
#     print("the dataframe before the columns were renamed")
    print(df.columns)
    df_not_renamed=df.copy()
#     df_not_renamed.drop("Open",inplace=True)
#     print(df_not_renamed.columns)
    # Define important variables:
    if isinstance(tickers, str):
        tickers = [tickers]
    if isinstance(cols, str):
        
        cols = [cols]
        print(cols)

    # For multi-level column indexing:
    if isinstance(df.columns.values[0], tuple):

        # Define important varibles
        columns = df.columns.values
        new_cols = []

        # Itarate through the multi-level column names and flatten them:
        for col in columns:
            temp = []
            for name in col:
                if name != '':
                    temp.append(name)
            new_temp = '_'.join(temp)
            new_cols.append(new_temp)
        
        # Change the column names:
        df.columns = new_cols
#         print(df.columns)
    
    # For uni-level colum indexing:
    elif isinstance(df.columns.values[0], str):
        
        # Define new names:
        col_names = [column+'_'+ticker for column in cols\
                     for ticker in tickers]
        df.columns = col_names
    
#     print("the dataframe after the columns have been renamed") 
#     print(df)

    return df

df2=pd.DataFrame(one_lvl_colnames(data,cols=columns,tickers=tickers))
df2_columns=list(df2.columns)
# print(df2.columns)
print(df2.head(5))
df2.columns = [col.replace('Adj Close', 'Close') for col in df2_columns]
# df2['close_act']=df2['Close']
# df2['Close']=df2['close_act'].shift(1)
# df2.fillna(method='ffill',inplace=True)
# df2.fillna(method='bfill',inplace=True)
df2.head(5)

[*********************100%%**********************]  1 of 1 completed
Index(['Adj Close', 'Volume', 'Open', 'High', 'Low'], dtype='object')
            Adj Close_DIS  Volume_DIS   Open_DIS   High_DIS    Low_DIS
Date                                                                  
2007-01-03      28.317095    13562595  33.748165  34.073711  33.531136
2007-01-04      28.540649     9806285  33.738300  34.083576  33.708706
2007-01-05      28.308821    10551445  33.807354  33.975060  33.531136
2007-01-08      28.565489     9479676  33.728436  34.162495  33.610054
2007-01-09      28.524092    11588444  34.034248  34.221684  33.491676


,Close_DIS,Volume_DIS,Open_DIS,High_DIS,Low_DIS
Date,,,,,
2007-01-03,28.317095,13562595,33.748165,34.073711,33.531136
2007-01-04,28.540649,9806285,33.738300,34.083576,33.708706
2007-01-05,28.308821,10551445,33.807354,33.975060,33.531136
2007-01-08,28.565489,9479676,33.728436,34.162495,33.610054
2007-01-09,28.524092,11588444,34.034248,34.221684,33.491676


In [6]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [11]:
import pandas as pd

def sma(data, n):
    sma_values = []
    for i in range(len(data)):
        if i >= n :
            sma = sum(data[i - n :i ]) / n
            sma_values.append(sma)
        else:
            sma_values.append(None)
            
#     sma_values=pd.DataFrame({'Data': data, 'SMA': sma_values})
    return sma_values

# # Example data (replace this with your own series)
# data = [1, 2, 4, 5, 6, 7, 8, 9, 10]

# # Define the period for SMA
# n = 3

# # Calculate SMA using the i-th value
# sma_values = sma(data, n)

# # Create a DataFrame to display results
# result_df = sma_values

# print(result_df)


In [14]:
from stockstats import StockDataFrame
def extract_and_wrap_ticker_data(df, columns):
    # Split column names into parts
    parts = [col.split('_') for col in columns]

    # Create a dictionary to store information about each ticker
    ticker_info = {}

    # Iterate through the parts list
    for part in parts:
        data_type = part[0]  # e.g., 'Adj Close', 'Close', etc.
        ticker = part[1]     # e.g., 'AAPL', 'GOOGL', etc.

        # Check if the ticker is already in the dictionary
        if ticker not in ticker_info:
            ticker_info[ticker] = {}

        # Add information for the ticker
        ticker_info[ticker][data_type] = df[f'{data_type}_{ticker}']

    # Create a dictionary to store stockstats DataFrames for each ticker
    ticker_dataframes = pd.DataFrame()
#     var_importance_all=

    # Iterate through ticker_info and wrap data with stockstats
    for ticker, data in ticker_info.items():
#         print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
        df = pd.DataFrame(data)
        print(ticker)
#         print(df)
        
#         print(stock_df)

#         stock_df.init_all()
        stock_df=df.copy()
        stock_df['close_10_sma']=sma(stock_df['Close'],2)
        stock_df['close_10_ema']=sma(stock_df['Close'],2)
        stock_df['close_-1_s']=sma(stock_df['Close'],2)
        

#         # Calculate RMSE for SMA
#         rmse_sma = np.sqrt(mean_squared_error(df['DIS_close'], df['DIS_close_10_sma']))
#         rmse_dict['SMA'] = rmse_sma

#         # Calculate RMSE for EMA
#         rmse_ema = np.sqrt(mean_squared_error(df['DIS_close'], df['DIS_close_10_ema']))
#         rmse_dict['EMA'] = rmse_ema

#         # Calculate RMSE for lagged values
#         rmse_lagged = np.sqrt(mean_squared_error(df['DIS_close'], df['DIS_close_-1_s']))
#         rmse_dict['Lagged'] = rmse_lagged
#         print("&&&&&&&&&&&&&&&&&&&&&&&&")
#         
        print(stock_df.columns)
        stock_df=stock_df[['Close', 'close_10_sma','close_10_ema', 'close_-1_s']]
        stock_df.columns = [ticker+"_" + col for col in pd.DataFrame(stock_df).columns]
        ticker_dataframes = pd.concat([ticker_dataframes,stock_df],axis=1)
        
    return ticker_dataframes

# Sample DataFrame with specified columns
columns = list(df2.columns)
# print(columns)



# Extract and wrap ticker data
ticker_dataframes = extract_and_wrap_ticker_data(df2, columns)

# Access information for a specific ticker (e.g., AAPL)
# print(ticker_dataframes)

ticker_dataframes

DIS
Index(['Close', 'Volume', 'Open', 'High', 'Low', 'close_10_sma',
       'close_10_ema', 'close_-1_s'],
      dtype='object')


,DIS_Close,DIS_close_10_sma,DIS_close_10_ema,DIS_close_-1_s
Date,,,,
2007-01-03,28.317095,NaN,NaN,NaN
2007-01-04,28.540649,NaN,NaN,NaN
2007-01-05,28.308821,28.428872,28.428872,28.428872
2007-01-08,28.565489,28.424735,28.424735,28.424735
2007-01-09,28.524092,28.437155,28.437155,28.437155
2007-01-10,28.557211,28.544790,28.544790,28.544790
2007-01-11,28.971205,28.540651,28.540651,28.540651
2007-01-12,29.153360,28.764208,28.764208,28.764208
2007-01-16,29.004328,29.062283,29.062283,29.062283


In [ ]:
def sma(data, n):
    sma_values = []
    for i in range(len(data)):
        if i >= n :
            sma = sum(data[i - n :i ]) / n
            sma_values.append(sma)
        else:
            sma_values.append(None)
            
#     sma_values=pd.DataFrame({'Data': data, 'SMA': sma_values})
    return sma_values

In [33]:
alpha=0.5
i=4
ema=4
alpha * data[i-1] + (1 - alpha) * ema

4.0

In [49]:
import pandas as pd

def ema_without_i(data, n):
    ema_values = []
    alpha = 2 / (n + 1)
#     ema = data[0]
    
    for i in range(len(data)):
#         print("&&&&&&&&&&&&&&&&&&&&&&&")
#         print(data[i])
#         print("&&&&&&&&&&&&&&&&&&&&&&")
        
        if i==n:
            ema=data[i-1]
#             print(ema)
            ema_values.append(ema)

            
        if i > n :
#             print(ema)
#             print(alpha)
#             print(data[i-1])
            ema = alpha * data[i-1] + (1 - alpha) * ema
#             print(ema)
#             print("*******************")
            ema_values.append(ema)
        else:
            ema_values.append(None)
            
        
    print(ema_values)
    return ema_values

# Example data (replace this with your own series)
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Define the period for EMA
n = 3

# Calculate EMA without using the i-th value
ema_values = ema_without_i(data, n)

# Create a DataFrame to display results
result_df = pd.DataFrame({'Data': data, 'EMA': ema_values})

print(result_df)


[None, None, None, 3, None, 3.5, 4.25, 5.125, 6.0625, 7.03125, 8.015625]


ValueError: All arrays must be of the same length